# Assignment 6

### Angelo Rosace

#### 1st Assignment statement

Program a function that given a causal graph (in matrix form) computes the
basis set of independences to be checked to assert if that graph is a feasible causal
structure.

For the purpose of writing this function we suppose to have an acyclic causal graph.

In [64]:
example_M = [[0,1,0,0,0,0],
             [0,0,1,1,0,0],
             [0,0,0,0,1,0],
             [0,0,0,0,1,0],
             [0,0,0,0,0,0],
             [0,0,0,0,1,0]]

example2_M = [[0,1,1,0,0,0],
             [0,0,0,0,0,0],
             [0,0,0,1,0,0],
             [0,0,0,0,1,1],
             [0,0,0,0,0,0],
             [0,0,0,0,0,0],
             [0,1,0,0,0,0]]

def get_parents(node, matrix):
    parents = []
    if node == 0:
        return []
    for i,col in enumerate(matrix):
        if col[node] == 1:
            parents.append(i)
    return parents
        
def basis_set(matrix):
    #non adjacent nodes => parents of the two vertices in the pair of non adjacent nodes
    basis = {}
    for i,row in enumerate(matrix):
        for j,col in enumerate(matrix[i]):
            if not col == 1 and not i >= j and sum(matrix[i]) > 0:
                non_adj_nodes = (i,j)
                parents_i = get_parents(i,matrix)
                parents_j = get_parents(j,matrix)
                parents= list(set(parents_i + parents_j))
                basis[non_adj_nodes] = parents
    return basis

basis_set = basis_set(example_M)
print(basis_set)
    

{(0, 2): [1], (0, 3): [1], (0, 4): [2, 3, 5], (0, 5): [], (1, 4): [0, 2, 3, 5], (1, 5): [0], (2, 3): [1], (2, 5): [1], (3, 5): [1]}


The output above represents the sets of undirected edges and their corresponding parents.

#### 2nd Assignment statement

Program a function that given a basis set and data for all the variables computes the independences given the data (using Pearson Correlation) and returns True if the independences are observed. The threshold of correlation to consider independent or dependent two variables is a parameter of the function.

The assignment asks us to use Pearson Correlation. In order to asses the significance of the pearson correlation coefficients that we are going to calculate we will apply a Student's t test.
If the variables, sampled randomly and measured on n units, are independent (the null hypothesis is that ⍴=0) then the following transformation will follow a Student’s t-distribution with n-2 degrees of freedom:

**t = [r*(n-2)^(1/2)] * [(1-r^2)^(1/2)]**

In [66]:
import numpy as np
from scipy.stats import pearsonr
import math
import pandas as pd

from scipy import stats, linalg

def create_data_set(number_of_variables,size):
    data = {}
    for i in range(number_of_variables):
        randnums = np.random.randint(1,10,size)
        data[i] = randnums
    return pd.DataFrame(data)

def condition(subsets,conditions):
    if not conditions:
        return subsets
    sub_data = []
    for subset in subsets:
        unique_values = list(set(list(subset[conditions[0]])))
        for value in unique_values:
            sub = subset[subset[conditions[0]] == value]
            if len(sub) >= 1:
                sub_data.append(sub)
    return condition(sub_data,conditions[1:])

def t_r(r,n):
    return (r*math.sqrt((n-2)))*((math.sqrt(1-(r**2))))     

def check_independence(basis_set, data, threshold):
    indep = False
    observed_indep = []
    for relation in basis_set:
        n = len(data[relation[0]])
        if len(basis_set[relation]) == 0:
            if len(data[relation[0]]) >= 2 and len(data[relation[1]]) >= 2:
                r = pearsonr(data[relation[0]],data[relation[1]])[0]
                t = t_r(r,n)
                if t < threshold:
                    indep = True
                observed_indep.append(indep)
        else:
            result = []
            sets = condition([data],basis_set[relation])
            r = 0
            overall_weigth = 0
            for s in sets:
                overall_weigth += len(s)
                if len(s[relation[0]]) >= 2 and len(s[relation[1]]) >= 2:
                    r += (pearsonr(s[relation[0]],s[relation[1]])[0])*len(s)
            weigthed_r = r/overall_weigth
            t = t_r(weigthed_r,n)
            if t < threshold:
                indep = True
            observed_indep.append(indep)
                
    if observed_indep == True:
        print("The independences are observed")
        return True
    else:
        print("The independences are not observed")
        return False

data_set = create_data_set(len(example_M),1000)
check_independence(basis_set,data_set,0.05)

data_set2 = create_data_set(len(example2_M),1000)
check_independence(basis_set,data_set,0.05)

The independences are not observed
The independences are not observed


False